In [29]:
import os
import pandas as pd
import openpyxl

def load_csvs_from_folders(root_folder):

    dataframes = {}

    subfolders = [
        folder for folder in os.listdir(root_folder) 
        if os.path.isdir(os.path.join(root_folder, folder))
    ]

    for subfolder in subfolders:
        subfolder_path = os.path.join(root_folder, subfolder)

        file_names = [
            f for f in os.listdir(subfolder_path) 
            if os.path.isfile(os.path.join(subfolder_path, f))
        ]

        for x, file_name in enumerate(file_names, start=1):
            file_path = os.path.join(subfolder_path, file_name)
            try:
                dataframes[f'{subfolder}_df_{x}'] = pd.read_csv(file_path)
            except Exception as e:
                print(f"Failed to load {file_name} in {subfolder}: {e}")

    return dataframes

root_folder = './ALLHOSTS - Copy/'  ### replace with your root folder path


dataframes = load_csvs_from_folders(root_folder)

print(f"\nLoaded DataFrames: ${dataframes['Linux_df_1'].iloc[0]}")




Loaded DataFrames: $Plugin ID                                                               10114
CVE                                                             CVE-1999-0524
CVSS v2.0 Base Score                                                      2.1
Risk                                                                      Low
Host                                                             10.46.254.70
Protocol                                                                 icmp
Port                                                                        0
Name                            ICMP Timestamp Request Remote Date Disclosure
Synopsis                    It is possible to determine the exact time set...
Description                 The remote host answers to an ICMP timestamp r...
Solution                    Filter out the ICMP timestamp requests (13), a...
See Also                                                                  NaN
Plugin Output               The difference 

In [30]:
def process_dataframes(dataframes, output_file):
    results = []

    for df_name, df in dataframes.items():
        if 'Risk' in df.columns:
            dataframes[df_name] = df[df['Risk'].notna()]
        else:
            print(f"'{df_name}' does not have a 'Risk' column, skipping...")

    for df_name, df in dataframes.items():
        required_columns = {'Name', 'Plugin Output', 'Solution', 'Host', 'Port', 'Risk', 'See Also', 'CVE', 'Description'}
        if required_columns.issubset(df.columns):
            # Use Plugin Output as the reference
            unique_outputs = df['Plugin Output'].unique()

            for plugin_output in unique_outputs:
                filtered_rows = df[df['Plugin Output'] == plugin_output]

                name = filtered_rows['Name'].iloc[0]  # Take the first name associated with this Plugin Output
                solution = filtered_rows['Solution'].iloc[0]
                risk = filtered_rows['Risk'].iloc[0]
                see_also = filtered_rows['See Also'].iloc[0]
                cve = ", ".join(filtered_rows['CVE'].dropna().unique())  # Combine unique CVEs
                description = filtered_rows['Description'].iloc[0]  # Take the first description

                hosts = []
                for _, row in filtered_rows.iterrows():
                    host = row['Host']
                    port = row['Port']
                    if port == 0:
                        hosts.append(host)  # Only include the Host if Port is 0
                    else:
                        hosts.append(f"{host}/{port}")  # Include both Host and Port if Port is not 0

                hosts_str = ", ".join(hosts)

                results.append({
                    'Name': name,
                    'Plugin Output': plugin_output,
                    'Solution': solution,
                    'Risk': risk,
                    'See Also': see_also,
                    'CVE': cve,
                    'Description': description,
                    'Hosts': hosts_str
                })
        else:
            print(f"'{df_name}' is missing required columns, skipping...")

    if results:
        results_df = pd.DataFrame(results)
        # Ensure rows with the same Plugin Output and other details are unique
        results_df = results_df.drop_duplicates(subset=['Plugin Output', 'Solution', 'Risk', 'See Also', 'CVE', 'Description'])

        for _, result in results_df.iterrows():
            print(f"Name: {result['Name']}")
            print(f"Plugin Output: {result['Plugin Output']}")
            print(f"Solution: {result['Solution']}")
            print(f"Risk: {result['Risk']}")
            print(f"See Also: {result['See Also']}")
            print(f"CVE: {result['CVE']}")
            print(f"Description: {result['Description']}")
            print(f"Hosts: {result['Hosts']}")
            print("-" * 50)

        results_df.to_excel(output_file, index=False)
        print(f"\nResults saved to '{output_file}'")
    else:
        print("No results to save.")

# Example Usage
process_dataframes(dataframes, 'output_results_unique.xlsx')


Name: ICMP Timestamp Request Remote Date Disclosure
Plugin Output: The difference between the local and remote clocks is -29824 seconds.

Solution: Filter out the ICMP timestamp requests (13), and the outgoing ICMP
timestamp replies (14).
Risk: Low
See Also: nan
CVE: CVE-1999-0524
Description: The remote host answers to an ICMP timestamp request.  This allows an
attacker to know the date that is set on the targeted machine, which
may assist an unauthenticated, remote attacker in defeating time-based
authentication protocols.

Timestamps returned from machines running Windows Vista / 7 / 2008 /
2008 R2 are deliberately incorrect, but usually within 1000 seconds of
the actual system time.
Hosts: 10.46.254.70
--------------------------------------------------
Name: SSH Weak Key Exchange Algorithms Enabled
Plugin Output: 
The following weak key exchange algorithms are enabled : 

  diffie-hellman-group-exchange-sha1

Solution: Contact the vendor or consult product documentation to disable 

In [31]:
def process_and_sort_excel(input_file, output_file):
    try:
        df = pd.read_excel(input_file)
        
        if 'Name' not in df.columns or 'Hosts' not in df.columns:
            print(f"'{input_file}' does not have the required columns, unable to process.")
            return
        
        df['Hosts'] = df['Hosts'].apply(lambda x: ', '.join(sorted(set(x.split(', ')))))

        sorted_df = df.sort_values(by='Name')
        
        print(f"Rows before saving to Excel: {len(sorted_df)}")
        
        sorted_df.to_excel(output_file, index=False)
        
        print(f"Rows after saving to Excel: {len(sorted_df)}")
        print(f"Sorted results saved to '{output_file}'")

    except Exception as e:
        print(f"Error processing the file: {e}")

process_and_sort_excel('output_results_unique.xlsx', 'sorted_results.xlsx')


Rows before saving to Excel: 519
Rows after saving to Excel: 519
Sorted results saved to 'sorted_results.xlsx'


In [ ]:
def format_excel_data(input_file, output_file):
    try:
        df = pd.read_excel(input_file)

        required_columns = ['Name', 'Plugin Output', 'Solution', 'Hosts', 'CVE', 'Description']
        if not all(col in df.columns for col in required_columns):
            print(f"'{input_file}' is missing one or more required columns.")
            return

        # Group by 'Solution'
        grouped = df.groupby('Solution')

        with open(output_file, 'w') as f:
            for solution, group in grouped:
                # Combine all unique 'Name', 'CVE', 'Description', and 'Hosts'
                names = ', '.join(sorted(group['Name'].unique()))
                cves = ', '.join(sorted(set(group['CVE'].dropna())))
                descriptions = '\n'.join(sorted(set(group['Description'])))
                hosts = ', '.join(sorted(set(host for host_list in group['Hosts'] for host in host_list.split(', '))))
                
                # Write grouped data to the file
                f.write(f"Name: {names}\n")
                f.write(f"Plugin Output: -\n")
                f.write(f"Solution: {solution}\n")
                f.write(f"CVE: {cves}\n")
                f.write(f"Description:\n{descriptions}\n")
                f.write(f"Affected Hosts: {hosts}\n\n")
                f.write("=" * 20 + "\n\n")

        print(f"Formatted results saved to '{output_file}'")

    except Exception as e:
        print(f"Error processing the file: {e}")

# Example usage
format_excel_data('sorted_results.xlsx', 'formatted_output.txt')



Formatted results saved to 'formatted_output.txt'
